In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns



In [186]:
def introduce_missing_values(df, missing_rate = 0.1):
    df_missing = df.copy()
    n_missing = int(np.floor(missing_rate*df.size))
    for _ in range(n_missing):
        i = np.random.randint(0,df.shape[0])
        j = np.random.randint(0,df.shape[1])
        df_missing.iloc[i,j] = np.nan
    return df_missing
    

In [ ]:
def define_target(csv,target, learning_method='Supervised'):
    df = pd.read_csv(csv)
    ### Remove in prod
    df = introduce_missing_values(df)
    if learning_method == 'Supervised':
        df.dropna(subset=[target],inplace=True)
    features = df.drop(target, axis = 1)
    target = df[target]
    return df, features , target

In [199]:
df, features, target = define_target("./telco.csv", 'Churn')

662
6381


In [204]:
def identify_numeric(df, inspect_columns =False):
    numeric_cols = []
    
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col].dropna()):
            numeric_cols.append(col)
            continue
        
        temp_conversion = pd.to_numeric(df[col], errors='coerce')
        numeric_count = temp_conversion.notna().sum()
        valid_count = df[col].notna().sum()
        numeric_ratio = numeric_count / valid_count 
        if valid_count > 0 and numeric_ratio > 0.95:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            numeric_cols.append(col)
            
    df[numeric_cols] = df[numeric_cols].dropna().apply(
    lambda x: x.astype('int64') if x.dropna().astype(float).apply(lambda v: v.is_integer()).all() else x.astype(float)
)
    
    if inspect_columns == True:
        print(f'Numeric cols are:', numeric_cols)
            
    return numeric_cols

In [205]:
numeric_cols = identify_numeric(df,inspect_columns=True)

Numeric cols are: ['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'categorical_test']


In [206]:
def identify_boolean(df):
    boolean_cols = []
    boolean_mappings = {}
    categorical_cols = []
    high_cardinality_cols = []
    
    
    for col in df.columns:
        
        unique_values = df[col].dropna().unique()
        normalized_values = [str(val).strip().lower() for val in unique_values]
        cardinality_ratio = len(unique_values) / len(df[col].dropna())
        num_na = df[col].isna().sum()
        
        if len(normalized_values) == 2 or (len(normalized_values) == 1 and num_na > 0 ):
            boolean_cols.append(col)
            sorted_values = sorted(unique_values, key=str)
            boolean_mappings[col] = {sorted_values[0]:0 , sorted_values[1]: 1}
            
            continue
        
        if len(unique_values) < 10:
            categorical_cols.append(col)
            continue

        if pd.api.types.is_numeric_dtype(df[col]):
            continue
        
        try:
            df[col] = pd.to_datetime(df[col].dropna(),errors='raise')
        except:
            pass
        
        if cardinality_ratio > 0.1:
            high_cardinality_cols.append(col)
            continue
        
        
        categorical_cols.append(col)
            
        
    return boolean_cols, boolean_mappings,high_cardinality_cols,categorical_cols

In [207]:
boolean_cols, boolean_mappings, high_cardinality_cols ,categorical_cols = identify_boolean(df)

In [209]:
def cleanup_numeric_cols (numeric_list, boolean_list, categorical_list):
    numerical_cleaned_bool = list(set(numeric_list) - set(boolean_list))
    numerical_cleaned_all = list(set(numerical_cleaned_bool) - set(categorical_list))
    return numerical_cleaned_all

In [210]:
numerical_cols = cleanup_numeric_cols(numeric_cols, boolean_cols, categorical_cols)

In [ ]:
df.dtypes

customerID           object
gender               object
SeniorCitizen       float64
Partner              object
Dependents           object
tenure              float64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
categorical_test    float64
dtype: object

In [214]:
numerical_cols

['tenure', 'MonthlyCharges', 'TotalCharges']

In [ ]:

def detect_outliers(df, numerical_cols):
    outlier_report = {}
    
    for col in numerical_cols:
        col_report = {}
        Q